# From HWGQ-Caffe to HLS: A simple FINN example

<span style="color:red">**WARNING: FINN is undergoing a major overhaul at the moment, so the information below (especially the implementation details) is subject to change.**</span>

We'll use a pre-trained, binarized network that comes with FINN. As we'll see in a moment, it receives 784 inputs (which is the size of one image in the MNIST dataset), transforms it with three layers of 256 neurons, then produces 10 outputs (one for each digit).

In [1]:
import os
FINN_ROOT = os.environ['FINN_ROOT']
params = FINN_ROOT + "/inputs/sfc-w1a1.caffemodel"
topology = FINN_ROOT + "/inputs/sfc-w1a1.prototxt"
! ls -l $params
! ls -l $topology

-rw-r--r-- 1 root root 1344691 Jun 11 14:22 /app/FINN/../FINN/FINN/inputs/sfc-w1a1.caffemodel
-rw-r--r-- 1 root root 2964 Feb 20 14:52 /app/FINN/../FINN/FINN/inputs/sfc-w1a1.prototxt


In [2]:
! head -n 58 $topology

name: "sfc-w1a1-hwgq"
input: "data"
input_shape {
  dim: 64
  dim: 1
  dim: 28
  dim: 28
}
layer {
  name: "bn_inp"
  type: "BatchNorm"
  bottom: "data"
  top: "bn_inp"
  param {
    lr_mult: 0
  }
  param {
    lr_mult: 0
  }
  param {
    lr_mult: 0
  }
  batch_norm_param {
    moving_average_fraction: 0.95
  }
}
layer {
  name: "qt_inp"
  type: "Quant"
  bottom: "bn_inp"
  top: "qt_inp"
  quant_param {
    forward_func: "sign"
    backward_func: "hard_tanh"
    clip_thr: 1.0
  }
}
layer {
  name: "ip1"
  type: "BinaryInnerProduct"
  bottom: "qt_inp"
  top: "ip1"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  inner_product_param {
    num_output: 256
    bias_term: false
    weight_filler {
      type: "gaussian"
      std: 0.005
    }
  }
  binary_inner_product_param {
    use_alpha: true
  }  
}


Here you can see how the input data is normalized and quantized, and the first binarized fully-connected (here called "inner product") layer. Tools such as [Netscope](https://ethereon.github.io/netscope/#/editor) can help understand the structure of the network.

## Frontend: From Caffe to FINN IR
The frontend stage is responsible for converting QNNs trained by a variety of frameworks to the FINN intermediate representation (IR). As each framework exposes their QNN topologies through custom formats, FINN must first perform a conversion to a common IR that it knows how to process.

In [3]:
# import a trained network through the HWGQ-Caffe frontend
import FINN.frontend.frontend_hwgq as fe
imported_net = fe.importCaffeNetwork(topology, params)
imported_net

[LinearLayer,
 BipolarThresholdingLayer,
 FullyConnectedLayer,
 LinearLayer,
 LinearLayer,
 BipolarThresholdingLayer,
 FullyConnectedLayer,
 LinearLayer,
 LinearLayer,
 BipolarThresholdingLayer,
 FullyConnectedLayer,
 LinearLayer,
 LinearLayer,
 BipolarThresholdingLayer,
 FullyConnectedLayer,
 LinearLayer,
 LinearLayer]

## What functionality does the IR expose?
Let's get acquainted with what the FINN IR looks like and try to look "inside" some of the layers. As FINN is open source you could simply look at the IR source code in `FINN/core/layers.py`, but here we'll use a little helper function to investigate the exposed members and functions dynamically.


In [4]:
def showMembers(layer):
    return filter(lambda x: not x.startswith("_"), dir(layer))

In the current version of FINN IR, we only support linear topologies (i.e. no skip connections, just one layer after another). As such, there is no graph representation and parent/child pointers; the IR is simply a list of objects that correspond to supported neural network layers. We can look at different layers by accessing this list.

In [5]:
imported_net[2]

FullyConnectedLayer

In [6]:
showMembers(imported_net[2])

['W',
 'execute',
 'getInputSize',
 'getNumOps',
 'getOutputSize',
 'getParamSize',
 'getTotalInputBits',
 'getTotalOutputBits',
 'getTotalParamBits',
 'get_filter_dim',
 'get_in_dim',
 'get_out_dim',
 'get_pad',
 'get_parallel',
 'get_stride',
 'get_type',
 'ibits',
 'in_dim',
 'insize',
 'kernel',
 'obits',
 'outsize',
 'updateBitwidths',
 'wbits']

In [7]:
print("Inputs: %d, outputs: %d" % (imported_net[2].getInputSize(), imported_net[2].getOutputSize()))

Inputs: 784, outputs: 256


In [8]:
print("Weight matrix shape: %s bits per weight: %d" % (str(imported_net[2].W.shape), imported_net[2].wbits))

Weight matrix shape: (256, 784) bits per weight: 1


In fact, large parts of the FINN IR are *executable*, as you probably guessed from the `execute` function. At any point, we can generate a random vector of appropriate dimensions and pass it through this layer by calling `execute` to see what the output would be like.

In [9]:
import numpy as np
rand_inp_vec = np.random.randn(784)
ret = imported_net[2].execute(rand_inp_vec)
ret.shape

(256,)

As expected, we get an output vector of the expected size when we pass in an input vector of appropriate size for this layer. Let's have a quick look at what other layers are defined in the FINN IR:

In [28]:
import FINN.core.layers as lb
filter(lambda x: x.endswith("Layer"), dir(lb))

['BipolarThresholdingLayer',
 'ChanDeinterleaveLayer',
 'ChanInterleaveLayer',
 'ConvolutionLayer',
 'DummyLayer',
 'ExternalExecutionLayer',
 'FullyConnectedLayer',
 'Layer',
 'LinearLayer',
 'MatrixThresholdLayer',
 'MonitorLayer',
 'PaddingLayer',
 'PoolingLayer',
 'ReLULayer',
 'SlidingWindowLayer',
 'SoftmaxLayer',
 'ThresholdingLayer',
 'isConvLayer',
 'isFCLayer',
 'isLinearLayer',
 'isMatrixLayer',
 'isMatrixThresholdLayer',
 'isMaxPoolingLayer',
 'isPoolingLayer',
 'isReLULayer',
 'isScalarLinearLayer',
 'isSoftmaxLayer',
 'isThresholdLayer']

## The Streamlining Transform

You may have noticed that our imported network contains a bunch of `LinearLayer` instances with floating point parameters (which is currently indicated in FINN IR as 32 bits), like this one:

In [11]:
print("Layer type: %s weight bits: %d" % (imported_net[3].get_type(), imported_net[3].wbits))

Layer type: LinearLayer weight bits: 32


So where did this come from in a binarized network? Many state-of-the-art BNN/QNN methods use some floating point computation in the forward pass to improve the accuracy. Some examples are batch normalization layers and channel-wise scaling factors. Although these layers do not typically contain a large amount of computation, they may still incur slowdowns on devices where floating point operations are expensive and increase the memory footprint of the QNN by adding floating point parameters. This is the case for creating "dataflow-style" FPGA accelerators, so we'd like to somehow get rid of those floating point parameters and operations. 

Fortunately, we can use what is referred to as [streamlining](https://arxiv.org/pdf/1709.04060.pdf) to do this, without losing any accuracy! Streamlining is implemented as a *transformation* in FINN: a function that takes in the FINN IR representation of a network, and returns a transformed FINN IR representation.

In [12]:
# example of a device-neutral transform: streamlining
import FINN.transforms.transformations as tf
streamlined_net = tf.makeCromulent(imported_net)
print(streamlined_net)
print("Number of layers in original imported network: %d" % len(imported_net))
print("Number of layers in streamlined network: %d" % len(streamlined_net))

[BipolarThresholdingLayer, FullyConnectedLayer, BipolarThresholdingLayer, FullyConnectedLayer, BipolarThresholdingLayer, FullyConnectedLayer, BipolarThresholdingLayer, FullyConnectedLayer, LinearLayer]
Number of layers in original imported network: 17
Number of layers in streamlined network: 9


You can see that all `LinearLayer`s besides the final one have disappeared. This is achieved by updating the thresholds of the network. You can read more about how this is done [here](https://arxiv.org/pdf/1709.04060.pdf), or by looking at the source code for this transformation.

In [13]:
print("Original input normalization coefficients: %s %s " % (str(imported_net[0].A), str(imported_net[0].B)))
print("Original input quantization threshold: " + str(imported_net[1].thresholds))
print("Streamlined input quantization threshold: " + str(streamlined_net[0].thresholds))

Original input normalization coefficients: [0.01270615] [-0.42501277] 
Original input quantization threshold: [[0.]]
Streamlined input quantization threshold: [[34]]


## Verification: Does it still work?

How do we know that the transformed network still does the correct thing, i.e. classify the MNIST dataset as well as the original imported one did? In order to check this, we can make use of the execution capabilities of FINN IR. We start by importing some functions from the `FINN.core.coverification` module that lets us load the MNIST dataset, and display the first test image:

In [14]:
import FINN.core.coverification as cov
(X,Y)= cov.loadMNIST()
sample_mnist_input=X[0][0]
sample_mnist_input.shape

(28, 28)

In [15]:
import PIL.Image
PIL.Image.fromarray(sample_mnist_input)

Now we can use the convenience function `FINN.core.nn.execPipeline` to check what the original imported and transformed IRs make out of this image:

In [16]:
import FINN.core.nn as nn
orig_res = nn.execPipeline(sample_mnist_input.flatten(), imported_net)[0]
streamlined_res = nn.execPipeline(sample_mnist_input.flatten(), streamlined_net)[0]

In [17]:
orig_res

array([-4.22489643e-03, -9.47696529e-01, -5.63656226e-01, -4.29998435e-01,
       -2.10290406e-01, -1.75468760e+00, -3.00302663e+00,  8.65674712e+00,
       -3.76429997e+00,  1.67605694e+00])

In [18]:
streamlined_res

array([-4.22489643e-03, -9.47696529e-01, -5.63656226e-01, -4.29998435e-01,
       -2.10290406e-01, -1.75468760e+00, -3.00302663e+00,  8.65674712e+00,
       -3.76429997e+00,  1.67605694e+00])

In [19]:
np.isclose(orig_res, streamlined_res).all()

True

In [20]:
print("Original prediction: %d streamlined prediction: %d" % (np.argmax(orig_res), np.argmax(streamlined_res)))

Original prediction: 7 streamlined prediction: 7


Note that streamlining *may* produce slightly different results for large/deep networks due to how floating point results are influenced by rounding and order of operations, this is why we use np.isclose instead of comparing with the equals operator. 

## From FINN IR to HLS: It's Transforms All the Way Down

FINN comes with a Vivado HLS library of components that lets us deploy QNNs as dataflow accelerators on FPGAs. We can use the FINN FPGA dataflow backend to generate an HLS implementation that makes use of this library. The FPGA dataflow backend actually has a convenience call `synthesize()` that handles all the intermediate steps of going from FINN IR to HLS, but we explicitly make step-by-step calls here to show the details of how the backend works. 

In [29]:
import inspect

def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))

We'll have a look at the `prepare` function that the FPGA dataflow backend exposes to see what kind of steps are taken to go from the device-neutral FINN IR all the way down to HLS.

In [30]:
showSrc(be.prepare)

def prepare(pipeline, ibits):
    "Convert given QNN into a form processable by the FPGA backend."
    # interleave channels
    pipeline = trns.apply_repeated(pipeline, trns.passInterleaveChannels)
    # fuse activations to easily detect matrix-threshold op pairs
    pipeline = trns.apply_repeated(pipeline, trns.passFuseActivations)
    # compute/update bitwidths, using the specified # of input bits
    myUpdateBitwidths = lambda x: trns.passUpdateBitwidths(x, ibits)
    pipeline = trns.apply_repeated(pipeline, myUpdateBitwidths)
    # convert all layers to their Caffe implementation variants
    pipeline = trns.apply_repeated(pipeline, passConvertToFPGALayers)
    # give names to each layer -- useful for e.g. interactive PE/SIMD setting
    pipeline = trns.apply_repeated(pipeline, trns.passGiveUniqueNames)
    # sanity check
    pipeline = trns.apply_repeated(pipeline, passCheckProduceConsume)
    return pipeline



We see that many transforms are applied in the process (and this is not all of them, you can see more in the `synthesize` function call). Every step brings the original IR description a little closer to the function calls available in the FINN dataflow HLS library. For instance, have a look at the `passFuseActivations` below.

In [31]:
showSrc(tf.passFuseActivations)

def passFuseActivations(pipeline):
    "Replace (Matrix, Threshold) layer pairs with fused equivalents."
    inStages = pipeline
    inStages.reverse()
    numChanges = 0
    ret = []
    while len(inStages) > 1:
        layerA = inStages.pop()
        layerB = inStages.pop()
        if lb.isMatrixLayer(layerA) and lb.isThresholdLayer(layerB):
            ret += [lb.MatrixThresholdLayer("", layerA, layerB)]
            numChanges += 1
        else:
            ret += [layerA]
            inStages.append(layerB)
    # pop final element, if any left
    if len(inStages) == 1:
        ret += [inStages.pop()]
    return (ret, numChanges)



This particular pass, as expressed in the docstring, simply replaces pairs of linear transform and thresholding layers with a single layer (`MatrixThresholdLayer`) that "contains" both. How does this help? The FINN dataflow HLS library exposes a primitive (the so-called MVTU, or Matrix-Vector-Threshold Unit) that implements these operations. By gradually translating the higher-level representation into something that resembles the HLS library call, we get closer to our goal. Let's look at the source for transform, the `passConvertToFPGALayers`.

In [32]:
showSrc(be.passConvertToFPGALayers)

def passConvertToFPGALayers(pipeline):
    "Convert supported layers to their FPGA dataflow layer equivalents."
    ret = []
    numChanges = 0
    for L in pipeline:
        if L.get_type().startswith("FPGA"):
            # already FPGA layer -- copy as-is
            ret += [L]
        elif layers_base.isMatrixThresholdLayer(L):
            if layers_base.isFCLayer(L.mlayer):
                # TODO the conditions need to be more specific (bipolar
                # weights)
                ret += [layers_fpga.FPGABipolarMatrixThresholdLayer(L)]
            elif layers_base.isConvLayer(L.mlayer):
                # TODO the conditions need to be more specific (bipolar
                # weights)
                ret += [layers_fpga.FPGABipolarConvThresholdLayer(L)]
            else:
                raise Exception("Unsupported matrix-threshold combination for FPGA backend")
            numChanges += 1
        elif layers_base.isFCLayer(L):
            # TODO the conditions need to be more

This pass is actually quite straightforward; it replaces instances of device-neutral FINN IR layers (such as `MatrixThresholdLayer`) with equivalent FPGA-backend-specific layers (such as `FPGAMatrixThresholdLayer`). If it encounters an unsupported layer it simply raises an exception. Let's now call the `prepare()` function and examine its output.

In [33]:
import FINN.backend.fpga.backend_fpga as be
part_for_synthesis = streamlined_net[1:-1]
fpga_dataflow_net = be.prepare(part_for_synthesis, ibits=1)
fpga_dataflow_net

[FPGABipolarMatrixThresholdLayer,
 FPGABipolarMatrixThresholdLayer,
 FPGABipolarMatrixThresholdLayer,
 FPGABipolarMatrixLayer]

In [34]:
showMembers(fpga_dataflow_net[0])

['bram_cost',
 'canIncreaseMMV',
 'canIncreasePE',
 'canIncreaseSIMD',
 'codegen_architecture',
 'codegen_declarations',
 'codegen_globals',
 'codegen_outstream2single',
 'codegen_params',
 'codegen_single2instream',
 'execute',
 'getAccWidth',
 'getIBits',
 'getIBufName',
 'getIGroup',
 'getInStreamDecl',
 'getInStreamW',
 'getInputElemDType',
 'getInputSize',
 'getInputStreamDType',
 'getMMV',
 'getMatrixDims',
 'getMaxMMV',
 'getMaxPE',
 'getMaxSIMD',
 'getNumInputElems',
 'getNumOps',
 'getNumOutputElems',
 'getOBits',
 'getOBufName',
 'getOGroup',
 'getOutStreamDecl',
 'getOutStreamW',
 'getOutputElemDType',
 'getOutputSize',
 'getOutputStreamDType',
 'getPE',
 'getParamFileName',
 'getParamSuffix',
 'getSIMD',
 'getT',
 'getTMemCount',
 'getTMemDType',
 'getTMemName',
 'getW',
 'getWBits',
 'getWMemCount',
 'getWMemDType',
 'getWMemName',
 'get_filter_dim',
 'get_in_dim',
 'get_out_dim',
 'get_pad',
 'get_parallel',
 'get_stride',
 'get_type',
 'ibits',
 'insize',
 'instreamW',
 

We can see that the functionality exposed by the `FPGABipolarMatrixThresholdLayer` is significantly different from that of the `MatrixThresholdLayer`: namely, the backend-specific layer type contains many functions and members that control the details of the implementation, such as the folding factors (how much time multiplexing is done across each axis) controlled by the `pe, simd, mmv` parameters. We also see that this layer exposes certain code generation functions like these:

In [24]:
print(fpga_dataflow_net[0].codegen_architecture())


FCLayer_Batch<64, 1, 1, 1, 16, 784, 256, 200704, 256>(inStream, FPGABufferLayer_1, weights_FPGABipolarMatrixThresholdLayer_0, thres_FPGABipolarMatrixThresholdLayer_0, numReps);


In [25]:
print(fpga_dataflow_net[0].codegen_declarations())


        #pragma HLS ARRAY_PARTITION variable=weights_FPGABipolarMatrixThresholdLayer_0 complete dim=1
        #pragma HLS ARRAY_PARTITION variable=thres_FPGABipolarMatrixThresholdLayer_0 complete dim=1
        
